## Setup

In [2]:
# Connect using pyodbc, sqlalchemy, and pandas
import sqlalchemy
import numpy as np
import pandas as pd

server = "sqlsvr-0092-mdp-02.85f8a2f57eaf.database.windows.net"
database = "Staging"
username = "pisrc-inkoo"
password = input("Enter database password: ")
driver = "ODBC Driver 17 for SQL Server"

engine = sqlalchemy.create_engine(
    f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"
)


def query_db(query, params=None):
    return pd.read_sql(query, engine, params=params)


df = query_db("SELECT @@version;")
print(df[""][0])


Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Aug 10 2022 15:14:09 
	Copyright (C) 2022 Microsoft Corporation



# Binge Experience & PDF Analytics

In [3]:
# Columns of interest
query_db(
    """
    SELECT TOP 20
        SessionVisitorId,
        VisitStartDateTime,
        EventList,
        PageURL,
        PageName,
        EloquaContactId,
        UTM_Source,
        UTM_Medium,
        UTM_Campaign,
        UTM_Content,
        UTM_Term
        BingeId,
        BingeName,
        BingeCampaignId,
        BingeCriticalScore,
        BingeScoredAsset,
        BingeScoredAssetPath,
        BingeScoredAssetScore,
        PDFurl,
        PDFtitle,
        PDFpagecount
    FROM aem.RawTraffic
    WHERE VisitStartDateTime > '2022-06-15'
        AND BingeId <> '';
    """
)


,SessionVisitorId,VisitStartDateTime,PageURL,PageName,EloquaContactId,UTM_Source,UTM_Medium,UTM_Campaign,UTM_Content,BingeId,BingeName,BingeCampaignId,BingeCriticalScore,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore,PDFurl,PDFtitle,PDFpagecount
0,583719550121999618_121251133910496911,2022-06-15 01:35:17,https://www.rockwellautomation.com/search/ra-z...,rockwellautomation:search,,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,null,121
1,7155208166619660672_5588629153287961608,2022-06-15 09:03:27,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,"Getting Started in IAB (85 minutes, 121 pages)",121
2,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
3,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
4,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
5,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
6,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
7,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
8,4405954664442376543_7269107232429611232,2022-06-15 13:41:42,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
9,4405954664442376543_7269107232429611232,2022-06-15 13:41:42,https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,


# Lead Status

In [4]:
# ra_leadstage possible values
query_db(
    """
    SELECT DISTINCT
        ra_leadstage,
        ra_leadstagename
    FROM crm.Lead
    ORDER BY ra_leadstage;
    """
)


,ra_leadstage,ra_leadstagename
0,NaN,None
1,1.0,Unassigned
2,2.0,Awaiting Tele Acceptance
3,3.0,Awaiting Tele Qualification
4,4.0,Awaiting Sales Acceptance
5,5.0,Awaiting Sales Qualification
6,6.0,Distributor Lead
7,7.0,Qualified
8,8.0,External Lead


In [5]:
# statecode possible values
query_db(
    """
    SELECT DISTINCT
        statecode,
        statecodename
    FROM crm.Lead
    ORDER BY statecode;
    """
)


,statecode,statecodename
0,0,Open
1,1,Qualified
2,2,Disqualified


In [7]:
# Get latest lead status
query_db(
    """
    SELECT TOP 5
        c1.emailaddress1,
        c1.ra_leadstage,
        c1.ra_leadstagename,
        c1.statecode,
        c1.statecodename,
        c1.statuscode,
        c1.statuscodename
    FROM crm.Lead AS c1
    INNER JOIN (
        SELECT
            emailaddress1,
            MAX(createdon) AS createdon
        FROM crm.Lead
        GROUP BY emailaddress1
    ) AS c2
    ON c1.emailaddress1 = c2.emailaddress1
        AND c1.createdon = c2.createdon;
    """
)


,emailaddress1,ra_leadstage,ra_leadstagename,statecode,statecodename,statuscode,statuscodename
0,sherman.wagner@silganpfc.com,4,Awaiting Sales Acceptance,0,Open,1,New
1,miguel.chavez2@sbdinc.com,4,Awaiting Sales Acceptance,0,Open,1,New
2,jpierce@wlgore.com,5,Awaiting Sales Qualification,2,Disqualified,953810014,Duplicate Lead
3,timothy.smith@adient.com,4,Awaiting Sales Acceptance,0,Open,1,New
4,johnpatrick@rdm.net,4,Awaiting Sales Acceptance,0,Open,1,New


# Joins

In [3]:
# aem.RawTraffic: provides analytics data (key: EloquaContactID)
# crm.Lead: provides lead status (key: emailaddress1)
# elq.Contact: links the other two together (has both EloquaContactId and EmailAddress)

df = query_db(
    """
    WITH cl AS (
        SELECT
            c1.emailaddress1,
            c1.ra_leadstage,
            c1.statecode
        FROM crm.Lead AS c1
        INNER JOIN (
            SELECT
                emailaddress1,
                MAX(createdon) AS createdon
            FROM crm.Lead
            GROUP BY emailaddress1
        ) AS c2
        ON c1.emailaddress1 = c2.emailaddress1
            AND c1.createdon = c2.createdon
    )

    SELECT DISTINCT
        ec.EmailAddress,
        cl.ra_leadstage,
        ar.BingeId,
        ar.BingeName,
        ar.BingeCampaignId,
        ar.BingeCriticalScore,
        ar.BingeScoredAsset,
        ar.BingeScoredAssetPath,
        ar.BingeScoredAssetScore
    FROM
        aem.RawTraffic AS ar,
        cl,
        elq.Contact AS ec
    WHERE ar.EloquaContactId = ec.EloquaContactId
        AND cl.emailaddress1 = ec.EmailAddress
        AND ar.VisitStartDateTime > '2022-06-12'
        AND ar.BingeId <> '';
    """
)
df.to_csv("binge_leads_2022_09_28.csv")
df


,EmailAddress,ra_leadstage,BingeId,BingeName,BingeCampaignId,BingeCriticalScore,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore
0,alec.stull@cmcmaterials.com,5,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
1,alex.fonseca@ingredion.com,5,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
2,andrewbriones@kennedyjenks.com,3,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
3,arosa@bama.com,5,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
4,arturo.quispe@incontrolgroup.com,4,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
...,...,...,...,...,...,...,...,...,...
65,Timothy.Crane@CarlisleCCM.com,2,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,"Getting Started in IAB (85 minutes, 121 pages)",/content/dam/rockwell-automation/sites/downloa...,1
66,Timothy.Crane@CarlisleCCM.com,2,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,"Using the Architecture View (30 minutes, 38 pa...",/content/dam/rockwell-automation/sites/downloa...,1
67,tisha.lane@adaptecsolutions.com,4,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
68,win_tsai@solomon.com.tw,5,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,


In [2]:
query_db(
    """
    SELECT DISTINCT ec.EmailAddress
    FROM
        aem.RawTraffic AS ar,
        elq.Contact AS ec
    WHERE ar.EloquaContactId = ec.EloquaContactId
        AND ar.VisitStartDateTime > '2022-06-12'
        AND ar.BingeId <> '';
    """
)


,EmailAddress
0,atanvir@mcg-pyle.com
1,cristobal.escoto@ecnautomation.com
2,emanueljmascotto@gmail.com
3,juancarlos.chavez@brp.com
4,krosberg@plusgroups.com
5,ksydness@ifpusa.com
6,lujones@rapiscansystems.com
7,malvarez@kuehnecompany.com
8,mandar.parashare@tetrapak.com
9,mansour.eletreby@ecam.fr


In [6]:
# Columns of interest
df = query_db(
    """
    SELECT TOP 20
        PageURL,
        PageName,
        EloquaContactId,
        BingeId,
        BingeName,
        BingeCampaignId,
        BingeCriticalScore,
        BingeScoredAsset,
        BingeScoredAssetPath,
        BingeScoredAssetScore,
        PDFurl,
        PDFtitle,
        PDFpagecount
    FROM aem.RawTraffic
    WHERE VisitStartDateTime > '2022-06-15'
        AND BingeId <> ''
        AND PageName <> 'rockwellautomation:search'
    """
)
df.to_csv("binge_data.csv")


In [12]:
df = query_db(
    """
    SELECT DISTINCT
        BingeId,
        BingeName,
        BingeScoredAsset,
        BingeScoredAssetPath,
        BingeScoredAssetScore
    FROM aem.RawTraffic
    WHERE VisitStartDateTime >= '2022-06-01'
    """
)
df.to_csv("binge_assets.csv")
df


,BingeId,BingeName,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore
0,,,Electric Vehicles And Smart Manufacturing - A ...,/content/dam/rockwell-automation/sites/downloa...,2
1,,,"Project Design Assistant (30 minutes, 30 pages)",/content/dam/rockwell-automation/sites/downloa...,1
2,be1650550449644850,Basic Functionality Labs,"Export a BOM to ProposalWorks (20 minutes, 25 ...",/content/dam/rockwell-automation/sites/downloa...,1
3,be1650550449644850,Basic Functionality Labs,"Using the Architecture View (30 minutes, 38 pa...",/content/dam/rockwell-automation/sites/downloa...,1
4,be1654108452838784,The Race to Electric Vehicle Production is Here,Electric Vehicles And Smart Manufacturing - A ...,/content/dam/rockwell-automation/sites/downloa...,2
5,be1650550449644850,Basic Functionality Labs,,,
6,be1650550449644850,Basic Functionality Labs,"Getting Started in IAB (85 minutes, 121 pages)",/content/dam/rockwell-automation/sites/downloa...,1
7,,,"Getting Started in IAB (85 minutes, 121 pages)",/content/dam/rockwell-automation/sites/downloa...,1
8,None,None,None,None,None
9,,,,,


In [10]:
df = query_db(
    """
    SELECT DISTINCT
        PageURL,
        BingeId,
        BingeName
    FROM aem.RawTraffic
    WHERE VisitStartDateTime >= '2022-06-01'
    AND BingeId <> ''
    """
)
df.to_csv("binge_pages.csv")
df


,PageURL,BingeId,BingeName
0,https://locator.rockwellautomation.com/,be1650550449644850,Basic Functionality Labs
1,https://locator.rockwellautomation.com/,be1654108452838784,The Race to Electric Vehicle Production is Here
2,https://locator.rockwellautomation.com/Distrib...,be1650550449644850,Basic Functionality Labs
3,https://locator.rockwellautomation.com/SalesOf...,be1650550449644850,Basic Functionality Labs
4,https://locator.rockwellautomation.com/SalesOf...,be1650550449644850,Basic Functionality Labs
5,https://locator.rockwellautomation.com/systemi...,be1650550449644850,Basic Functionality Labs
6,https://locator.rockwellautomation.com/Technology,be1650550449644850,Basic Functionality Labs
7,https://locator.rockwellautomation.com/Technol...,be1650550449644850,Basic Functionality Labs
8,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs
9,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs


In [13]:
df = query_db(
    """
    SELECT DISTINCT
        EloquaContactId,
        BingeId,
        BingeName
    FROM aem.RawTraffic
    WHERE VisitStartDateTime >= '2022-06-01'
    AND BingeId <> ''
    """
)
df.to_csv("binge_visitors.csv")
df


,EloquaContactId,BingeId,BingeName
0,CRACP000015821975,be1650550449644850,Basic Functionality Labs
1,CRACP000015792782,be1650550449644850,Basic Functionality Labs
2,,be1654108452838784,The Race to Electric Vehicle Production is Here
3,CRACP000015824307,be1650550449644850,Basic Functionality Labs
4,CRACP000014727216,be1650550449644850,Basic Functionality Labs
5,CRACP000015557611,be1650550449644850,Basic Functionality Labs
6,CRACP000015821031,be1650550449644850,Basic Functionality Labs
7,CRACP000012736740,be1650550449644850,Basic Functionality Labs
8,CRACP000015738365,be1650550449644850,Basic Functionality Labs
9,CRACP000015716018,be1650550449644850,Basic Functionality Labs


In [17]:
df = query_db(
    """
    SELECT DISTINCT
        EloquaContactId,
        BingeScoredAsset
    FROM aem.RawTraffic
    WHERE VisitStartDateTime >= '2022-06-01'
        AND BingeId <> ''
    ORDER BY EloquaContactId
    """
)
df


,EloquaContactId,BingeScoredAsset
0,,"Using the Architecture View (30 minutes, 38 pa..."
1,,"Export a BOM to ProposalWorks (20 minutes, 25 ..."
2,,
3,,Electric Vehicles And Smart Manufacturing - A ...
4,CRACP000000213285,
5,CRACP000000218618,
6,CRACP000000774122,"Getting Started in IAB (85 minutes, 121 pages)"
7,CRACP000000774122,
8,CRACP000012736740,
9,CRACP000014727216,


In [22]:
df = query_db(
    """
    SELECT
        SessionVisitorId,
        VisitStartDateTime,
        EventList,
        PageURL,
        PageName,
        EloquaContactId,
        BingeId,
        BingeName,
        BingeCampaignId,
        BingeCriticalScore,
        BingeScoredAsset,
        BingeScoredAssetPath,
        BingeScoredAssetScore,
        PDFurl,
        PDFtitle,
        PDFpagecount
    FROM aem.RawTraffic
    WHERE VisitStartDateTime > '2022-06-01'
        AND BingeId <> '';
    """
)
df.to_csv("all_binge_data.csv")
df


,SessionVisitorId,VisitStartDateTime,EventList,PageURL,PageName,EloquaContactId,BingeId,BingeName,BingeCampaignId,BingeCriticalScore,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore,PDFurl,PDFtitle,PDFpagecount
0,1348294982902173390_3333818272057931229,2022-06-13 03:36:24,"215,216=1100.00,10035,10036,10037,10038,10039,113",https://locator.rockwellautomation.com/Technol...,rockwellautomation:locator:technology:productd...,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,"Applying Pricing to BOM (20 minutes, 15 pages)",15
1,1348294982902173390_3333818272057931229,2022-06-13 03:36:24,"213,214,10035,10036,10037,10038,10039,113",https://locator.rockwellautomation.com/Technol...,rockwellautomation:locator:technology:productd...,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,"Applying Pricing to BOM (20 minutes, 15 pages)",15
2,1348294982902173390_3333818272057931229,2022-06-13 03:36:24,"215,10035,10036,10037,10038,10039,113",https://locator.rockwellautomation.com/Technol...,rockwellautomation:locator:technology:productd...,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,"Applying Pricing to BOM (20 minutes, 15 pages)",15
3,7808339790048566800_7369824390511291230,2022-06-13 05:46:52,"215,216=32.00,10035,10036,10037,10038,10039,113",https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,null,121
4,1008979344765208242_653748637528310500,2022-06-13 07:20:12,"213,214,10035,10036,10037,10038,10039,113",https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,"Getting Started in IAB (85 minutes, 121 pages)",121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,981125255383133615_371879973670522003,2022-06-19 14:57:17,"215,216=1082.00,10035,10036,10037,10038,10039,113",https://locator.rockwellautomation.com/systemi...,rockwellautomation:locator:systemintegrator,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,"Using the Architecture View (30 minutes, 38 pa...",/content/dam/rockwell-automation/sites/downloa...,1,/content/dam/rockwell-automation/sites/downloa...,"Using the Architecture View (30 minutes, 38 pa...",38
498,981125255383133615_371879973670522003,2022-06-19 14:57:17,"213,214,10035,10036,10037,10038,10039,113",https://locator.rockwellautomation.com/systemi...,rockwellautomation:locator:systemintegrator,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,"Using the Architecture View (30 minutes, 38 pa...",/content/dam/rockwell-automation/sites/downloa...,1,/content/dam/rockwell-automation/sites/downloa...,"Using the Architecture View (30 minutes, 38 pa...",38
499,981125255383133615_371879973670522003,2022-06-19 14:57:17,"206,10020,10022,10023,10024,10025,10027,10029,...",https://locator.rockwellautomation.com/systemi...,rockwellautomation:locator:systemintegrator,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,"Using the Architecture View (30 minutes, 38 pa...",/content/dam/rockwell-automation/sites/downloa...,1,/content/dam/rockwell-automation/sites/downloa...,null,38
500,981125255383133615_371879973670522003,2022-06-19 14:57:17,"215,216=2.00,10035,10036,10037,10038,10039,113",https://locator.rockwellautomation.com/systemi...,rockwellautomation:locator:systemintegrator,,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,"Using the Architecture View (30 minutes, 38 pa...",/content/dam/rockwell-automation/sites/downloa...,1,/content/dam/rockwell-automation/sites/downloa...,null,38


In [8]:
df = query_db(
    """
    SELECT *
    FROM aem.RawTraffic
    WHERE VisitStartDateTime > '2022-06-12'
        AND SessionVisitorId LIKE '402319239204006%';
    """
)
df


,SessionVisitorId,VisitStartDateTime,VisitPageNumber,VisitNumber,NewVisit,EventList,ExcludeHit,HitSource,DateTime_UTC,PageURL,...,PDFurl,PDFtitle,PDFpagecount,BingeId,BingeName,BingeCriticalScore,BingeCampaignId,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore
